In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import pickle
from pathlib import Path

DATA_FOLDER = Path("../../data")
MODEL_FOLDER = Path("./models")

In [15]:
df = pd.read_parquet(DATA_FOLDER / "green_tripdata_2021-01.parquet")
df.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object

In [16]:
df["duration"] = (
    df.lpep_dropoff_datetime - df.lpep_pickup_datetime
).dt.total_seconds() / 60

In [17]:
df.duration.describe(percentiles=[0.95, 0.98, 0.99])

count    76518.000000
mean        19.927896
std         59.338594
min          0.000000
50%         13.883333
95%         44.000000
98%         56.000000
99%         67.158167
max       1439.600000
Name: duration, dtype: float64

In [18]:
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9658903787344154

In [19]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape

(73908, 21)

In [20]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]
df[categorical] = df[categorical].astype(str)

In [21]:
dv = DictVectorizer()

train_dicts = df[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

In [22]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',

DictVectorizer transforms data stored in a dictionary into a vector, in this case, a sparse matrix. It recognizes string fields and one-hot encodes them. It also recognizes numerical fields, and leaves them untouched. If we had not converted PULocation and DOLocation to strings, it would have treated them as numerical fields, which would be wrong for our application (as location numbers are only IDs, there is no notion of operation between them).

By checking the values stored in the resulting matrix one can see that except for the duration column, all the other values are 0s and 1s, consistent with the one-hot encoding. The feature names are also consistent with one-hot encoding for the `DOLocationID` and `PULocationID` fields, as every possible value originates a different column.

In [23]:
np.unique(X_train.toarray()[:, :-1].flatten())

array([0., 1.])

In [24]:
target = "duration"
y_train = df[target].values

In [25]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [26]:
y_pred = lm.predict(X_train)

In [27]:
root_mean_squared_error(y_train, y_pred)

9.838799799829625

The model predictions have an RMSE of 9.5 minutes. We can probably do better, but this is still the training data. We want to compute scoring metrics on validation data too. Since we will have to repeat pretty much the same steps, we put everything together in functions for convenience.

We are building _preprocessing and training pipelines._

In [28]:
def read_dataframe(filename):
    categorical = ["PULocationID", "DOLocationID"]

    df = pd.read_parquet(filename)

    df["duration"] = (
        df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    ).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[categorical] = df[categorical].astype(str)

    return df

In [29]:
# Read training and validation dataframes
df_train = read_dataframe(DATA_FOLDER / "green_tripdata_2021-01.parquet")
df_val = read_dataframe(DATA_FOLDER / "green_tripdata_2021-02.parquet")
df_train.shape, df_val.shape

((73908, 21), (61921, 21))

In [30]:
# Engineer an interaction feature
# Interaction features combine two (or more) features into one. The idea is that the pair of values matters more than the single values together.
# In our case we will create an interaction feature from PU and DO. The interpretation is that instead of capturing what is the marginal variation due to "pick up here" and "drop off there" separately, we'll try to capture the information carried by "pick up here and drop off there".
# Note that this also reduces in half the number of features, reducing the variance of the models.
interaction = True

# In this case it suffices to combine the string to create a new categorical feature
if interaction:
    df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
    df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [31]:
# Use DictVectorizer to encode categorical features and obtain training and validation matrices
if interaction:
    # When we use the interaction features, we drop the individual features.
    categorical = ["PU_DO"]
else:
    categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [32]:
# Obtain training and validation targets
target = "duration"
y_train = df_train[target]
y_val = df_val[target]

In [33]:
# Train and validate a linear regression model
lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred = lm.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.758715204520257

In [34]:
# Train and validate a Lasso model
ls = Lasso(alpha=0.001)
ls.fit(X_train, y_train)

y_pred = ls.predict(X_val)
root_mean_squared_error(y_val, y_pred)

9.233436225720547

In [35]:
# Train and validate a Ridge model
lr = Ridge(alpha=0.1)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.524898164485006

The validation error when we include pick-up and drop-off separately is about 10.5 minutes for the best models. When we replace them with the interaction feature, it decreases to about 7.5 minutes.

In [36]:
# Save the model for later use
with open(MODEL_FOLDER / "lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lm), f_out)